<div align="center">

# Projeto Final - Redes Complexas (2025)  
## **Professor:** Francisco A. Rodrigues  
## **Alunos:**  
Gabriel Freitas Ximenes de Vasconcelos - 11819084

Gustavo Gabriel Ribeiro - 13672683  

Letícia Barbanera Menezes - 14588642

</div>


### Complex Networks: An Introduction

Complex systems are composed of multiple units, generally simple in their structure, behavior, and individual functioning. These units interact with each other dynamically, and from these interactions emerge collective phenomena with characteristics that cannot be reduced to the mere sum of their parts.

A widely discussed example — including in Albert-László Barabási’s book *Linked* — is that of human social networks. Although each individual (or “node”) exhibits relatively simple behaviors — such as initiating a friendship or ceasing interaction with someone — their connections form a highly structured network, whose global patterns (such as *clusters*, *hubs*, and short paths) reveal surprising organizational properties.

Beyond this classical case, complex networks have applications in various fields:

- **Marketing**: It is possible to predict product adoption by modeling the audience as a complex system, allowing for strategy optimization based on connections between profiles, preferences, and influence.
- **Recommender systems**: Products, users, and the interactions between them can be modeled as a network, whose structure helps identify similarities, preferences, and potential suggestions.
- **Organizational logistics**: Units of production, storage centers, transportation routes, and consumer markets can be modeled as a network, where flows can be optimized based on the network’s topological structure.

Since complex phenomena are largely determined by how the system’s units are connected, the most suitable computational representation for these systems is the **graph**. Graphs are well-established mathematical structures, widely used in computer science to efficiently and flexibly represent nodes (entities) and edges (connections).


### Link Prediction: Why It Matters

Given that these systems are dynamic, one crucial task is to predict the emergence of new connections between pairs of individuals — a task known as **link prediction**.

This project aims to explore and compare several link prediction techniques proposed in the literature. The goal is to understand which algorithms perform best for a specific type of network — in this case, **a social network** — and investigate the possible reasons behind their performance.

To better illustrate the potential of this technique, consider a marketing-related example. Previous research in social networks and complex systems shows that, to maximize the **adoption of a product**, it is more effective to focus dissemination efforts on **specific communities** within the network, rather than trying to increase the campaign’s overall reach.

To do this, it is necessary to identify **strategic individuals**, i.e., those whose influence spreads most efficiently **within a target audience**. Interestingly, these individuals are **not necessarily the most popular influencers** (those with the most followers or connections), but rather those who occupy structurally advantageous positions in the network.

Moreover, **link prediction** allows us to anticipate **which new connections are likely to form** in the future. This insight is essential for forecasting **network evolution**, identifying emerging communities, and dynamically adjusting campaigns or recommendations.


# Installs

In [ ]:
%%capture
!pip install pandas numpy networkx matplotlib seaborn tqdm requests plotly

In [ ]:
import plotly.io as pio

# Imports

In [ ]:
import math
import random
from collections import defaultdict
from copy import deepcopy

import community as community_louvain
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import time
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats

pio.templates.default = "plotly_white"
import urllib.request

import networkx as nx
import requests
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from tqdm import tqdm
from xgboost import XGBClassifier

# Exploring the Network

## Basic Functions


In [ ]:
def draw_graph(graph,  fig_title: str, spring_layout: bool = False, figsize=(10, 8)):
    plt.figure(figsize=figsize)
    ax = plt.gca()

    if spring_layout:
        pos = nx.spring_layout(graph, seed=42)
    else:
        pos = nx.kamada_kawai_layout(graph)

    degrees = dict(graph.degree())
    node_colors = [degrees[node] for node in graph.nodes()]

    nodes = nx.draw(
        graph,
        pos=pos,
        with_labels=True,
        node_size=600,
        font_size=12,
        node_color=node_colors,
        cmap=plt.cm.viridis,
        edge_color="gray",
        linewidths=1,
        font_weight="bold",
        ax=ax,
    )

    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.viridis, norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors))
    )
    sm.set_array([])
    plt.colorbar(sm, ax=ax, shrink=0.7, label="Node Degree")

    plt.title(f"Graph Visualization: {fig_title}")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_interactive_network(G, title="Interactive Network Graph", save_fig: bool = False):
    pos = nx.spring_layout(G, seed=42)  #node position

    #edges' coordinates
    edge_x = []
    edge_y = []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    #nodes' edges and labels
    node_x = []
    node_y = []
    node_text = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(G.nodes[node].get('label', str(node)))

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        hoverinfo='text',
        textposition="top center",
        marker=dict(
            showscale=False,
            color='lightblue',
            size=10,
            line_width=1
        )
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=title,
                        title_x=0.5,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(visible=False),
                        yaxis=dict(visible=False)
                    ))

    fig.show()

    if save_fig:
        print("Saving Figure...")
        fig.write_html(f"./{title}.html")

In [ ]:
def network_basic_info(graph, fig_title: str, save_fig : bool = False):
    N = len(graph)
    M = graph.number_of_edges()
    density = nx.density(graph)
    is_directed = graph.is_directed()

    print("--- Basic Network Info ---")
    print("Directed" if is_directed else "Undirected")
    print("Number of nodes:", N)
    print("Number of edges:", M)
    print("Density:", round(density, 4))

    if nx.is_connected(graph) if not is_directed else nx.is_weakly_connected(graph):
        diameter = nx.diameter(graph)
        print("Diameter:", diameter)
        avg_shortest_path = nx.average_shortest_path_length(graph)
        print("Average shortest path length:", "%3.4f" % avg_shortest_path)
    else:
        print("Graph is not connected — diameter undefined.")

    if is_directed:
        num_weak_components = nx.number_weakly_connected_components(graph)
        num_strong_components = nx.number_strongly_connected_components(graph)
        print("Number of weakly connected components:", num_weak_components)
        print("Number of strongly connected components:", num_strong_components)
    else:
        num_components = nx.number_connected_components(graph)
        print("Number of connected components:", num_components)

    degrees = dict(graph.degree())
    avg_degree = sum(degrees.values()) / N
    max_deg_node = max(degrees, key=degrees.get)
    min_deg_node = min(degrees, key=degrees.get)

    print("Average degree:", round(avg_degree, 2))
    print("Maximum degree:", degrees[max_deg_node], "| Node:", max_deg_node)
    print("Minimum degree:", degrees[min_deg_node], "| Node:", min_deg_node)


## Loading network

In [ ]:
file_url = "https://raw.githubusercontent.com/GustavoZiel/complex-networks/refs/heads/main/Informatica%20%26%20Uporab.%20Inform./iui.net"

response = urllib.request.urlopen(file_url)
graph = nx.read_pajek(response, encoding='utf-8')
G = nx.Graph(graph)

print("Network carregada com sucesso!")

Network carregada com sucesso!


## Basic Information

In [ ]:
# Number of nodes
num_nos = G.number_of_nodes()
print(f"Número de Nós (Autores): {num_nos}")

# Number of edges
num_arestas = G.number_of_edges()
print(f"Número de Arestas (Coautorias): {num_arestas}")

# Whether or not the system is directed
is_directed = nx.is_directed(G)
tipo_rede = "Direcionada" if is_directed else "Não Direcionada"
print(f"Tipo da Rede: {tipo_rede}")

# Wheter or not the system is connected
if not is_directed:
    is_connected = nx.is_connected(G)
    status_conexao = "Conectada" if is_connected else "Não Conectada (Possui múltiplos componentes)"
    print(f"Status de Conexão: {status_conexao}")
    num_componentes = nx.number_connected_components(G)
    print(f"Número de Componentes Conectados: {num_componentes}")
else:
    is_connected = nx.is_weakly_connected(G)
    status_conexao = "Fracamente Conectada" if is_connected else "Desconectada"
    print(f"Status de Conexão: {status_conexao}")
    num_componentes = nx.number_weakly_connected_components(G)
    print(f"Número de Componentes Fracamente Conectados: {num_componentes}")

# Find largest connected component
if not is_connected and num_componentes > 0:
    largest_cc = max(nx.connected_components(G), key=len) if not is_directed else max(nx.weakly_connected_components(G), key=len)
    G_lc = G.subgraph(largest_cc)
    print(f"Tamanho do Maior Componente Conectado: {G_lc.number_of_nodes()} nós e {G_lc.number_of_edges()} arestas.")
else:
    G_lc = G

# Average node degree (number of connections)
grau_medio = sum(dict(G_lc.degree()).values()) / G_lc.number_of_nodes()
print(f"Grau Médio: {grau_medio:.2f}")

# Network's density
densidade = nx.density(G_lc)
print(f"Densidade da Rede: {densidade:.4f}")

Número de Nós (Autores): 2288
Número de Arestas (Coautorias): 3377
Tipo da Rede: Não Direcionada
Status de Conexão: Não Conectada (Possui múltiplos componentes)
Número de Componentes Conectados: 704
Tamanho do Maior Componente Conectado: 553 nós e 1234 arestas.
Grau Médio: 4.46
Densidade da Rede: 0.0081


The basic network information shows that this is a relatively small network, with 2,288 nodes. On average, each node has 4.46 connections (edges), totaling 3,377 edges. The network density is very low (0.0081), meaning that among all possible pairs of nodes, only a small fraction are actually connected by a collaboration — a typical characteristic of social networks.

The network is disconnected, meaning it is composed of multiple isolated components. In total, there are 704 connected components, which highlights the presence of several isolated groups of nodes. The largest connected component contains 553 nodes and 1,234 edges, representing a significant portion of the network, but still indicating a high level of fragmentation.



In [ ]:
# Distance
print("Distância:")
if not is_connected: # Non-connected network
    print(f"A rede não é conectada.")
    if G_lc.number_of_nodes() > 1:
        try:
            print(f"Calculando a distância média do caminho mínimo para o maior componente conectado (com {G_lc.number_of_nodes()} nós).")
            avg_shortest_path = nx.average_shortest_path_length(G_lc)
            print(f"Distância Média do Caminho Mínimo (Maior Componente): {avg_shortest_path:.2f}")
        except nx.NetworkXError as e:
            print(f"Não foi possível calcular o caminho médio mínimo: {e}")
            avg_shortest_path = float('inf')
    else:
        avg_shortest_path = float('inf')
        print("Maior componente conectado tem 1 nó ou menos. Caminho médio não aplicável.")
else: # Fully connected network
    if G.number_of_nodes() > 1:
        avg_shortest_path = nx.average_shortest_path_length(G)
        print(f"Distância Média do Caminho Mínimo (Rede Inteira): {avg_shortest_path:.2f}")
    else:
        avg_shortest_path = float('inf')
        print("Rede tem 1 nó ou menos. Caminho médio não aplicável.")

# Diameter
print("Diâmetro:")
if not is_directed and nx.is_connected(G_lc):
    diametro = nx.diameter(G_lc)
    print(f"Diâmetro da Rede: {diametro}")
elif is_directed and nx.is_strongly_connected(G_lc):
    diametro = nx.diameter(G_lc)
    print(f"Diâmetro da Rede (fortemente conectada): {diametro}")
else:
    print("Diâmetro não pode ser calculado (a rede não é fortemente/conectada).")


Distância:
A rede não é conectada.
Calculando a distância média do caminho mínimo para o maior componente conectado (com 553 nós).
Distância Média do Caminho Mínimo (Maior Componente): 7.23
Diâmetro:
Diâmetro da Rede: 20


In the largest connected component of the network, which includes 553 nodes, the average shortest path length is 7.23. This means that, on average, it takes about 7.23 connection “hops” to connect any two nodes within this component. The diameter of this component — that is, the longest shortest path between any two nodes — is 20, indicating that in the worst-case scenario, up to 20 hops are needed for an indirect collaboration between two nodes to occur.


## Visualization

In [ ]:
#@title Visualização da Rede com Plotly

pos = nx.spring_layout(G, seed=42)# k ajusta a distância, iterations a "estabilização"

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, line=dict(width=0.5, color="gray"), hoverinfo="none", mode="lines"
)

node_x = []
node_y = []
node_text = []
node_degree = []

for node, adjacencies in G.adjacency():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"Nó: {node}<br>Grau: {len(adjacencies)}")
    node_degree.append(len(adjacencies))


node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode="markers",
    hoverinfo="text",
    text=node_text,
    marker=dict(
        showscale=True,
        colorscale="YlGnBu",
        reversescale=True,
        color=node_degree,
        size=10,
        colorbar=dict(
            thickness=15,
            title="Grau do Nó",
            xanchor="left",
        ),
        line_width=2,
    ),
)

fig_rede = go.Figure(data=[edge_trace, node_trace])
fig_rede.update_layout(
    title_text="Visualização da Rede de Coautoria",
    title_x=0.5,
    xaxis_title="",
    yaxis_title="",
    showlegend=False,
)

fig_rede.update_layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)

fig_rede.show()

In [ ]:
most_connected_node = max(G.degree, key=lambda x: x[1])[0]
max_degree = G.degree[most_connected_node]
print(f"Fun Fact, o Sr. {most_connected_node} é o autor com maior número de coautorias, com {max_degree}.")

Fun Fact, o Sr. M. Gams é o autor com maior número de coautorias, com 38.


## Analysis of degree distribution of network

### Gráfico de Barras e Log-Log da Distribuição de Graus

In [ ]:
# Degree distribution of network

# frequency of each degree
degrees = [G.degree(n) for n in G.nodes()]
degree_counts = pd.Series(degrees).value_counts().sort_index()
mean_degree = np.mean(degrees)

# histogram
fig_dist_graus = go.Figure(
    data=[
        go.Bar(
            x=degree_counts.index, y=degree_counts.values, marker_color="rgb(26, 118, 255)", name=""
        )
    ]
)
fig_dist_graus.update_layout(
    title="Distribuição de Graus da Rede",
    title_x=0.5,
    xaxis_title="Grau (k)",
    yaxis_title="Número de Nós (Frequência)",
    showlegend=False,
)
fig_dist_graus.show()

# plotting distribution in logarithimic scale to improve visualization
fig_dist_graus_log = go.Figure(
    data=[
        go.Scatter(
            x=degree_counts.index,
            y=degree_counts.values,
            mode="markers+lines",
            marker_color="rgb(50, 171, 96)",
            name="",
        )
    ]
)
fig_dist_graus_log.update_layout(
    title="Distribuição de Graus da Rede (Escala Log-Log)",
    title_x=0.5,
    xaxis_title="Log(Grau (k))",
    yaxis_title="Log(Número de Nós)",
    showlegend=False,
)
fig_dist_graus_log.update_layout(xaxis_type="log", yaxis_type="log")
fig_dist_graus_log.show()

The bar chart clearly shows a right-skewed asymmetry, with a long tail to the right. This indicates that most nodes have few connections (nodes with low degree are predominant): the highest bar is at degree *k = 1* (around 800–850 individuals), meaning that the majority of unities in the network collaborated with only one other entity. This is followed by nodes with degree 2 (approximately 580), degree 3 (around 480), and so on. The number of nodes decreases rapidly as the degree increases.

In the lower plot, which uses a log-log scale, we observe that the green points tend to form a downward-sloping curve, but does not follow a perfectly straight line. This suggests that the distribution shows signs of following a **power-law trend**, although it does not strictly adhere to this pattern.


### Box-Plot dos Graus da Rede

In [ ]:
# Box Plot for network's degree
fig_boxplot_graus = go.Figure(
    data=[
        go.Box(
            y=degrees, name="Graus", marker_color="rgb(26, 118, 255)", boxpoints="outliers"
        )
    ]
)
fig_boxplot_graus.update_layout(
    title="Box Plot da Distribuição de Graus",
    title_x=0.5,
    yaxis_title="Grau"
)
fig_boxplot_graus.show()

Because this network has a high concentration of individuals with few connections, the boxplot is skewed toward lower values. As a result, a node with more than 4 links is already considered an outlier.


### Momentos da Rede

In [ ]:
# List of nodes' degrees
degrees = [d for n, d in G_lc.degree()]

# 1º Moment (Average)
mean_degree = np.mean(degrees)
print(f"1º Momento (Média): {mean_degree:.2f}")

# 2º Moment (Variance)
variancia_graus = np.var(degrees)
print(f"2º Momento (Variância): {variancia_graus:.2f}")

# 3º Moment (Skewness)
assimetria_graus = stats.skew(degrees)
print(f"3º Momento (Assimetria): {assimetria_graus:.2f}")

# 4º Moment (Kurtosis)
curtose_graus = stats.kurtosis(degrees)
print(f"4º Momento (Curtose): {curtose_graus:.2f}")

1º Momento (Média): 4.46
2º Momento (Variância): 22.09
3º Momento (Assimetria): 3.09
4º Momento (Curtose): 12.52


The moments of the network reveal some important insights (some of which were already evident in the plots) about the degree distribution of the nodes.

The **1st moment** represents the mean degree, indicating that, on average, each node has 4.46 co-authorships. The **2nd moment**, corresponding to the degree variance, is quite high (22.09), reflecting a strong heterogeneity in the network — while most nodes have few connections, a few have a significantly larger number of connections.

The **3rd moment**, which measures the skewness of the degree distribution, has a value of 3.09, confirming the strong right-skewed asymmetry observed in the degree bar chart: many nodes have few connections, while a few have an extremely high number of connections - those are called hubs, and are represented by those famous influencers.

Finally, the **4th moment**, which measures kurtosis, is highly positive (12.52), indicating the presence of outliers in the network — that is, nodes with degrees that are substantially different from the majority, as also seen in the boxplot.


## Network Clustering

### Average Clustering Coefficient and Transitivity

In [ ]:
# Coeficiente de Agrupamento e Transitividade

# Coeficiente de Agrupamento Médio
avg_clustering_coeff = nx.average_clustering(G)
print(f"Coeficiente de Agrupamento Médio: {avg_clustering_coeff:.4f}")

# Transitividade (Global Clustering Coefficient)
transitivity = nx.transitivity(G)
print(f"Transitividade (Coeficiente de Agrupamento Global): {transitivity:.4f}")

Coeficiente de Agrupamento Médio: 0.5608
Transitividade (Coeficiente de Agrupamento Global): 0.5243


When analyzing the **average clustering coefficient** of the network — a **local indicator** for each node — it is found that, on average, about 52.96% of an individual's friends also know each other. In other words, when two people, A and B, both are friends with a third person C, there is approximately a 52.96% chance that A and B have also knwon each other before.

On the other hand, the **transitivity**, which measures the **global** probability that two connections of the same node are also connected with each other, indicates a 62.98% chance for this to occur across the entire network.

These relatively high values reveal that the network is highly clustered, suggesting that people tend to organize themselves into cohesive communities, where individuals are strongly interconnected.

(This scenario can be observed by zooming in on the network plot, where several small groups of tightly connected nodes can be identified.)


### Network's Assortativity

In [ ]:
# Assortividade
assortativity_coeff = nx.degree_assortativity_coefficient(G)
print(f"Coeficiente de Assortividade por Grau: {assortativity_coeff:.4f}")

if assortativity_coeff > 0.1:
    print("Interpretação: A rede é assortativa. Autores tendem a colaborar com outros autores que têm um número similar de coautores. (Ex: pesquisadores prolíficos colaboram entre si).")
elif assortativity_coeff < -0.1:
    print("Interpretação: A rede é disassortativa. Autores com muitos coautores tendem a colaborar com aqueles que têm poucos coautores.")
else:
    print("Interpretação: A rede é aproximadamente neutra em termos de assortividade por grau. Não há uma preferência clara na conexão baseada no grau dos autores.")

Coeficiente de Assortividade por Grau: 0.2326
Interpretação: A rede é assortativa. Autores tendem a colaborar com outros autores que têm um número similar de coautores. (Ex: pesquisadores prolíficos colaboram entre si).


Additionally, the network exhibits a weak to moderate degree of assortativity, with a value of 0.2326.

This metric measures the tendency of nodes in a network to connect with other nodes that have a similar degree. In the context of social media and marketing, a positive assortativity value (such as 0.2326) indicates that highly connected users tend to engage with other highly connected users, while less connected individuals interact mostly within their own low-degree circles. This structure suggests the presence of tightly-knit influencer communities, which can limit the organic spread of information across the broader network. For marketing strategies, this implies that targeting only top influencers may not be sufficient for wide reach; instead, bridging different audience segments becomes essential to ensure effective diffusion of content or campaigns.



### Local Clustering Coefficients' Distribution

In [ ]:
local_clustering_coeffs = nx.clustering(G)
clustering_values = list(local_clustering_coeffs.values())

# Parâmetros do histograma
use_probability_density = False
use_probability = True

if use_probability_density:
    histnorm_param = 'probability density'
    yaxis_title_text = 'Densidade de Probabilidade'
else:
    # Para usar probabilidade (soma das alturas das barras = 1) em vez de densidade:
    histnorm_param = 'probability'
    yaxis_title_text = 'Probabilidade (Proporção de Nós)'
    # Para usar contagem bruta:
    histnorm_param = None
    yaxis_title_text = 'Frequência (Número de Nós)'


fig_dist_clustering_improved = px.histogram(
    x=clustering_values,
    nbins=30,
    histnorm=histnorm_param,
    marginal="rug",
    opacity=0.75,
    color_discrete_sequence=['#2E86C1'],
    labels={'x': 'Coeficiente de Agrupamento Local'},
)

fig_dist_clustering_improved.update_layout(
    title='Distribuição dos Coeficientes de Agrupamento Locais',
    xaxis_title='Coeficiente de Agrupamento Local',
    title_x=0.5,
    yaxis_title=yaxis_title_text
)
fig_dist_clustering_improved.update_traces(
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5
)

# Calcular e adicionar linhas para média e mediana
if clustering_values:
    mean_coeff = np.mean(clustering_values)
    median_coeff = np.median(clustering_values)

    fig_dist_clustering_improved.add_vline(
        x=mean_coeff,
        line_width=2,
        line_dash="dash",
        line_color="green",
        annotation_text=f"Média: {mean_coeff:.3f}",
        annotation_position="top right",
        annotation_font_size=10,
        annotation_font_color="green"
    )
    fig_dist_clustering_improved.add_vline(
        x=median_coeff,
        line_width=2,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mediana: {median_coeff:.3f}",
        annotation_position="bottom right",
        annotation_font_size=10,
        annotation_font_color="red"
    )

fig_dist_clustering_improved.show()

It is possible to observe that the network is highly polarized at both extremes in terms of local clustering coefficients, with values concentrated at 0 and 1.

A local clustering coefficient of 0 indicates that an individual's friends have no connections among themselves. On the other hand, a coefficient of 1 means that all of that node's followers also follow each other, forming a fully connected group.

The median clustering coefficient reinforces this observation, suggesting that it is common in this network to find groups of users who collaborate intensely with each other, forming tightly connected communities — or, more informally, **cliques**.


### Average Clustering Coefficient by nodes' degree

In [ ]:
# Clustering Coefficient por Grau

node_ids = list(G.nodes())
degrees_list = [G.degree(n) for n in node_ids]
clustering_list = [local_clustering_coeffs[n] for n in node_ids]

df_degree_clustering = pd.DataFrame(
    {"degree": degrees_list, "clustering_coefficient": clustering_list}
)

avg_clustering_by_degree = (
    df_degree_clustering.groupby("degree")["clustering_coefficient"].mean().reset_index()
)

fig_clust_by_degree = go.Figure(
    data=[
        go.Scatter(
            x=avg_clustering_by_degree["degree"],
            y=avg_clustering_by_degree["clustering_coefficient"],
            mode="markers",
            marker=dict(size=10, color="purple"),
            name="Média do Coef. de Agrupamento",
        )
    ]
)

fig_clust_by_degree.add_trace(
    go.Scatter(
        x=df_degree_clustering["degree"],
        y=df_degree_clustering["clustering_coefficient"],
        mode="markers",
        marker=dict(size=5, color="rgba(210, 0, 210, 0.2)"),
        name="Coef. de Agrupamento Individual",
        hoverinfo="skip",
    )
)


fig_clust_by_degree.update_layout(
    title="Coeficiente de Agrupamento Médio por Grau",
    xaxis_title="Grau (k)",
    yaxis_title="Coeficiente de Agrupamento Médio C(k)",
)
fig_clust_by_degree.show()

The plot above shows how the average local clustering coefficient varies as a function of node degree.

It is clear that C(k) (vertical axis) decreases as k increases (horizontal axis), indicating that users with many friends tend to have neighbors who are less interconnected. In contrast, higher values of C(k), associated with lower degrees (between 3 and 10), suggest that **cliques**—tightly connected groups of friends—are more common among nodes with a smaller number of collaborations.

This decreasing trend of C(k) may also indicate the presence of **hierarchical structure** within the network. Users with many friends (high-degree nodes) tend to act as **hubs** or connection's centers, connecting different groups that are not directly linked to each other, which results in lower local clustering. On the other hand, among users with fewer friends, such hierarchical organization is not observed: their neighbors are often connected to one another, forming locally dense clusters.


### largest Clique

In [ ]:
import itertools

# Encontrar todos os cliques maximais
cliques = list(nx.find_cliques(G))
maior_panelinha_nodes = max(cliques, key=len)
tamanho_maior_panelinha = len(maior_panelinha_nodes)
print(f"A maior 'panelinha' encontrada tem {tamanho_maior_panelinha} nós.")

# Coletar nós e arestas da maior panelinha
maior_panelinha_nodes_set = set(maior_panelinha_nodes)
maior_panelinha_edges_set = set()
for u, v in itertools.combinations(maior_panelinha_nodes, 2):
    if G.has_edge(u, v):
        edge = tuple(sorted((u, v)))
        maior_panelinha_edges_set.add(edge)

# Arestas da maior panelinha
maior_panelinha_edge_x = []
maior_panelinha_edge_y = []
for u, v in maior_panelinha_edges_set:
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    maior_panelinha_edge_x.extend([x0, x1, None])
    maior_panelinha_edge_y.extend([y0, y1, None])

maior_panelinha_edge_trace = go.Scatter(
    x=maior_panelinha_edge_x,
    y=maior_panelinha_edge_y,
    line=dict(width=2.5, color='red'),
    hoverinfo='none',
    mode='lines',
    name=f'Maior Panelinha ({tamanho_maior_panelinha} Nós)'
)

# Arestas restantes
other_edge_x = []
other_edge_y = []
for u, v in G.edges():
    edge = tuple(sorted((u, v)))
    if edge not in maior_panelinha_edges_set:
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        other_edge_x.extend([x0, x1, None])
        other_edge_y.extend([y0, y1, None])

other_edge_trace = go.Scatter(
    x=other_edge_x,
    y=other_edge_y,
    line=dict(width=0.5, color='#cccccc'),
    hoverinfo='none',
    mode='lines',
    name='Outras Arestas'
)

# Nós
node_colors_list = []
node_sizes_list = []
node_text_list = []

for node in G.nodes():
    degree = G.degree(node)
    text = f'Nó: {node}<br>Grau: {degree}'
    if node in maior_panelinha_nodes_set:
        node_colors_list.append('red')
        node_sizes_list.append(12)
        text += '<br><b>Na Maior Panelinha</b>'
    else:
        node_colors_list.append('rgba(100, 100, 150, 0.7)')
        node_sizes_list.append(8)
    node_text_list.append(text)

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text_list,
    marker=dict(
        color=node_colors_list,
        size=node_sizes_list,
        line=dict(width=1, color='black')
    ),
    name='Nós'
)

# Construção da figura
fig_maior_panelinha = go.Figure(data=[
    other_edge_trace,
    maior_panelinha_edge_trace,
    node_trace
])

fig_maior_panelinha.update_layout(
    title=f'Visualização da Rede com Destaque para a Maior Panelinha ({tamanho_maior_panelinha} Nós)',
    title_x=0.5,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig_maior_panelinha.show()

A maior 'panelinha' encontrada tem 11 nós.


Based on the plots presented, we identify here the largest **clique** in the network—that is, the most strongly connected group of people.

Despite being the largest one found, this clique is not particularly large, containing only 11 users. This reinforces the idea that, although cohesive communities do exist in the network, they tend to be relatively small in size.


## Analysis of Communities within the network

In [ ]:
#@title Plot

if G.number_of_nodes() > 0 and G.number_of_edges() > 0:
    try:
        communities_generator = nx.community.louvain_communities(G, seed=42)
        communities_list = sorted(list(communities_generator), key=len, reverse=True)
        num_total_communities = len(communities_list)
        print(f"Número Total de Comunidades Encontradas (Louvain): {num_total_communities}")

        if num_total_communities > 0:
            # Para melhorar visualização, focamos em no máximo 10 comunidades
            num_top_communities_to_show = 10
            if num_total_communities <= num_top_communities_to_show:
                num_top_communities_to_show = num_total_communities

            print(
                f"Mostrando as {num_top_communities_to_show} maiores comunidades individualmente."
            )
            print(
                f"Tamanhos das {num_top_communities_to_show} primeiras comunidades: {[len(c) for c in communities_list[:num_top_communities_to_show]]}"
            )

            modularity = nx.community.modularity(G, communities_list)
            print(f"Modularidade das Comunidades Encontradas: {modularity:.4f}")

            viz_partition_map = {}
            for i, comm_nodes in enumerate(communities_list):
                if i < num_top_communities_to_show:
                    for node in comm_nodes:
                        viz_partition_map[node] = i
                else:
                    for node in comm_nodes:
                        viz_partition_map[node] = num_top_communities_to_show

            node_colors_viz = [
                viz_partition_map.get(node, num_top_communities_to_show) for node in G.nodes()
            ]

            if num_top_communities_to_show > 0:
                colors_for_top_n = px.colors.qualitative.Plotly[:num_top_communities_to_show]
            else:
                colors_for_top_n = []

            custom_colorscale = colors_for_top_n + ["lightgrey"]

            tickvals_viz = list(range(num_top_communities_to_show + 1))
            ticktext_viz = [f"Com. {i}" for i in range(num_top_communities_to_show)] + [
                "Outras Com."
            ]

            if num_total_communities == 0:
                node_colors_viz = ["grey"] * G.number_of_nodes()
                custom_colorscale = ["grey"]
                tickvals_viz = [0]
                ticktext_viz = ["N/A"]

            node_trace_comm_improved = go.Scatter(
                x=node_x,
                y=node_y,
                mode="markers",
                hoverinfo="text",
                text=[
                    f'Nó: {n}<br>Comunidade (Grupo Viz): {viz_partition_map.get(n, "Outras")}'
                    for n in G.nodes()
                ],
                marker=dict(
                    # showscale=True,
                    colorscale=custom_colorscale,
                    color=node_colors_viz,
                    cmin=0,
                    cmax=num_top_communities_to_show,
                    size=10,
                    colorbar=dict(
                        thickness=15,
                        title="Grupo de Comunidade",
                        xanchor="left",
                        tickvals=tickvals_viz,
                        ticktext=ticktext_viz,
                        lenmode="fraction",
                        len=0.75,
                    ),
                    line_width=1,
                ),
            )

            fig_rede_comm_improved = go.Figure(data=[edge_trace, node_trace_comm_improved])
            fig_rede_comm_improved.update_layout(
                title=f"Visualização da Rede com Top {num_top_communities_to_show} Comunidades (Louvain)",
                xaxis_title="",
                title_x=0.5,
                yaxis_title="",
                showlegend=False,
            )
            fig_rede_comm_improved.update_layout(
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            )
            fig_rede_comm_improved.show()

        else:
            print("Nenhuma comunidade encontrada.")

    except Exception as e:
        print(f"Erro ao detectar ou visualizar comunidades: {e}")
        communities_list = []
        modularity = None
else:
    print("Rede muito pequena ou sem arestas para detecção de comunidades significativa.")
    communities_list = []
    modularity = None

Número Total de Comunidades Encontradas (Louvain): 714
Mostrando as 10 maiores comunidades individualmente.
Tamanhos das 10 primeiras comunidades: [112, 97, 70, 58, 48, 41, 35, 33, 27, 26]
Modularidade das Comunidades Encontradas: 0.9699


We applied the Louvain algorithm for community detection in the network and obtained a very interesting result. A total of 714 distinct communities were identified, with the 10 largest highlighted in the plot. The modularity of the resulting partition is 0.96 — an extremely high value, indicating a very well-defined division of the network into cohesive groups.

Modularity typically ranges between -0.5 and 1. Higher values, usually above 0.3 or 0.4, suggest a good separation between communities. Therefore, the value of 0.96 strongly reinforces that users are organized into well-defined groups, with many connections within each community and relatively few between different communities.


# Link Prediction

### Link Prediction Approaches

We tested two different approaches to predict links in the network:

#### 1. Unsupervised (Heuristic-Based)

In this approach, we simply apply formulas that give a score to each pair of unconnected nodes. The higher the score, the more likely it is that a link should exist.

#### 2. Supervised (Machine Learning Classifiers)

Here, we treat link prediction as a binary classification problem:

- **Positive examples (class 1):** node pairs that had a real link in the original network, but were temporarily removed to test the model.
- **Negative examples (class 0):** random node pairs that don’t have a link.

**Steps:**

1. Remove a small percentage of links (e.g., 10%) from the original network — these become the test set (positives).
2. Keep the remaining links as the training set (positives).
3. Sample an equal number of unconnected node pairs to act as negative examples (for both training and testing).
4. Build a table (dataframe) with node pairs and their structural features (like Common Neighbors, Jaccard, etc.).
5. Train a model using these features to learn patterns.
6. The model receives node pair features and predicts the **probability** that a link exists between them.


## Step 1

In [ ]:
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)

### Splitting the Edges into Train and Test Sets (Before Extracting Any Features!)

- Take all the edges (links) from the original network.
- Randomly remove a portion (e.g., 15%) to create the **test set** (these are the positive examples).
- The remaining edges form the **training graph**.
- Then, generate **negative samples** for both train and test: random pairs of nodes that are not connected in the graph.

### Creating a DataFrame with Node Pair Features

- For each pair of nodes (whether truly connected or not — i.e., labeled as 1 or 0), extract:
  - **Pairwise features**, based on their relationship in the graph.
  - **Individual node features**, which are then combined.


In [ ]:
def split_edges_train_test(G, test_size=0.15, seed=42):

    # Divide a rede G em duas versões: uma com arestas de treino (G_train)
    # e outra com as arestas de teste separadas (test_edges).
    # Remove 15% das arestas para teste, garantindo que G_train continue conectado.

    random.seed(seed)

    G_train = deepcopy(G)
    all_edges = list(G.edges())
    num_test = int(len(all_edges) * test_size)
    random.shuffle(all_edges)

    test_edges = []
    for edge in all_edges:
        u, v = edge
        G_train.remove_edge(u, v)
        if nx.is_connected(G_train):
            test_edges.append(edge)
            if len(test_edges) >= num_test:
                break
        else:
            G_train.add_edge(u, v)

    return G_train, test_edges

In [ ]:
G_train, test_edges = split_edges_train_test(G, test_size=0.15)

print("Arestas no treino:", len(G_train.edges()))
print("Arestas no teste:", len(test_edges))

Arestas no treino: 985
Arestas no teste: 173


In [ ]:
def generate_negative_edges(G, num_samples, excluded_edges=set(), seed=42):

    # Gera pares de nós (u, v) que não estão conectados em G.
    # Serve para o modelo aprender a diferenciar pares de nós que tem arestas e que não tem
    # Caso contrário, ele só ia aprender casos em que há arestas e,
    # quando confrontado com um caso em que não há aresta, não ia saber o que fazer

    random.seed(seed)
    nodes = list(G.nodes())
    negative_edges = set()

    while len(negative_edges) < num_samples:
        u, v = random.sample(nodes, 2)
        if (u, v) in G.edges() or (v, u) in G.edges():
            continue
        if (u, v) in excluded_edges or (v, u) in excluded_edges:
            continue
        if (u, v) in negative_edges or (v, u) in negative_edges:
            continue
        negative_edges.add((u, v))

    return list(negative_edges)

In [ ]:
train_edges = list(G_train.edges())

In [ ]:
train_negatives = generate_negative_edges(G_train, len(train_edges), excluded_edges=set(train_edges + test_edges))
test_negatives = generate_negative_edges(G_train, len(test_edges), excluded_edges=set(train_edges + test_edges + train_negatives))

In [ ]:
#gerando uma amostra balançeada
print("Positivos treino:", len(train_edges))
print("Negativos treino:", len(train_negatives))
print("Positivos teste:", len(test_edges))
print("Negativos teste:", len(test_negatives))

Positivos treino: 985
Negativos treino: 985
Positivos teste: 173
Negativos teste: 173


In [ ]:
#algumas medidas frequentemente usadas para pares de vértices
from networkx.algorithms.link_prediction import (
    jaccard_coefficient,
    adamic_adar_index,
    resource_allocation_index
)

In [ ]:
def build_feature_dataframe(G, edge_list, label):

    #G: grafo (G_train!)
    #edge_list: lista de pares (u, v) — positivos ou negativos
    #cria as colunas do dataframe. cria um dicionário pra cada medida, onde a chave é a aresta e o valor é uma lista com cada medida
    #depois coloca essas medidas no dataframe. atenção para uv = vu !

    df = pd.DataFrame(edge_list, columns=['node1', 'node2'])
    df['common_neighbors'] = 0
    df['jaccard'] = 0.0
    df['adamic_adar'] = 0.0
    df['resource_allocation'] = 0.0
    df['label'] = label


    cn_dict = {(u, v): len(list(nx.common_neighbors(G, u, v))) for u, v in edge_list}
    jaccard_dict = {(u, v): p for u, v, p in jaccard_coefficient(G, edge_list)}
    adamic_dict = {(u, v): p for u, v, p in adamic_adar_index(G, edge_list)}
    ra_dict = {(u, v): p for u, v, p in resource_allocation_index(G, edge_list)}


    for i, row in df.iterrows():
        u, v = row['node1'], row['node2']
        df.at[i, 'common_neighbors'] = cn_dict.get((u, v), cn_dict.get((v, u), 0))
        df.at[i, 'jaccard'] = jaccard_dict.get((u, v), jaccard_dict.get((v, u), 0))
        df.at[i, 'adamic_adar'] = adamic_dict.get((u, v), adamic_dict.get((v, u), 0))
        df.at[i, 'resource_allocation'] = ra_dict.get((u, v), ra_dict.get((v, u), 0))

    return df

In [ ]:
df_train_pos = build_feature_dataframe(G_train, train_edges, label=1)
df_train_neg = build_feature_dataframe(G_train, train_negatives, label=0)
df_train = pd.concat([df_train_pos, df_train_neg]).sample(frac=1, random_state=42).reset_index(drop=True)
df_test_pos = build_feature_dataframe(G_train, test_edges, label=1)
df_test_neg = build_feature_dataframe(G_train, test_negatives, label=0)
df_test = pd.concat([df_test_pos, df_test_neg]).sample(frac=1, random_state=42).reset_index(drop=True)

print(df_train.head())
print(df_test.head())

   node1  node2  common_neighbors   jaccard  adamic_adar  resource_allocation  \
0    275    383                 1  0.333333     0.402430             0.083333   
1    288    422                 1  0.071429     0.314658             0.041667   
2    130    196                 0  0.000000     0.000000             0.000000   
3    283    533                 0  0.000000     0.000000             0.000000   
4    404     56                 0  0.000000     0.000000             0.000000   

   label  
0      1  
1      1  
2      0  
3      0  
4      0  
   node1  node2  common_neighbors   jaccard  adamic_adar  resource_allocation  \
0     20    244                 1  0.083333     0.314658             0.041667   
1    336    391                 1  0.125000     0.910239             0.333333   
2    536    152                 1  0.500000     0.352956             0.058824   
3    246    196                 0  0.000000     0.000000             0.000000   
4    533    226                 0  0.00000

In [ ]:
print(df_train.shape)
print(df_test.shape)

(1970, 7)
(346, 7)


In [ ]:
def add_node_features(G, df):

    # Adiciona features baseadas nas métricas individuais dos nós:
    # - grau_soma -> a importância é acumulativa
    # - closeness_soma -> a importância é acumulativa
    # - betweenness_max -> basta um deles ser forte
    # - eigenvector_prod -> efeitos são conjuntos
    # - kcore_min -> é preciso que ambos sejam fortes
    # - clustering_mean -> se ambos formam triângulos, é um bom sinal, pois maior chance de formar ali tb
    #funcionamento semelhante à função anterior: cria um dicionário cuja chave é o nó e o valor é a respectiva medida
    #depois adiciona no dataframe combinando as medidas da forma descrita para cada aresta

    degree_dict = dict(G.degree())
    closeness_dict = nx.closeness_centrality(G)
    betweenness_dict = nx.betweenness_centrality(G)
    eigenvector_dict = nx.eigenvector_centrality(G, max_iter=1000)
    kcore_dict = nx.core_number(G)
    clustering_dict = nx.clustering(G)

    df['grau_soma'] = df.apply(lambda row: degree_dict[row['node1']] + degree_dict[row['node2']], axis=1)
    df['closeness_soma'] = df.apply(lambda row: closeness_dict[row['node1']] + closeness_dict[row['node2']], axis=1)
    df['betweenness_max'] = df.apply(lambda row: max(betweenness_dict[row['node1']], betweenness_dict[row['node2']]), axis=1)
    df['eigenvector_prod'] = df.apply(lambda row: eigenvector_dict[row['node1']] * eigenvector_dict[row['node2']], axis=1)
    df['kcore_min'] = df.apply(lambda row: min(kcore_dict[row['node1']], kcore_dict[row['node2']]), axis=1)
    df['clustering_mean'] = df.apply(lambda row: (clustering_dict[row['node1']] + clustering_dict[row['node2']]) / 2, axis=1)

    return df


In [ ]:
df_train = add_node_features(G_train, df_train)
df_test = add_node_features(G_train, df_test)

In [ ]:
print(df_train.shape)
print(df_test.shape)

(1970, 13)
(346, 13)


## Step 2

In [ ]:
# List with the metrics obtained from each model
results = []

### Common Neighbors Based

The idea: *"If two nodes share many neighbors, it's likely they will connect in the future."*  
This is a **heuristic** !

#### Unsupervised Approach:

- For a new node pair (u, v), we calculate a similarity score
- We then compare this score to the scores of all other pairs in the network.
- We look for the most similar node pairs. We observe the **labels** of these similar node pairs (1 for real links, 0 for non-existing links).
- Based on this, we make a decision:  
  If most of the similar pairs are real links, then the new pair is **likely** to be a real link too.

#### Common Neighbors Based - Common Neighbors Algorithm (CN) - Baseline

In [ ]:
df_train.columns

Index(['node1', 'node2', 'common_neighbors', 'jaccard', 'adamic_adar',
       'resource_allocation', 'label', 'grau_soma', 'closeness_soma',
       'betweenness_max', 'eigenvector_prod', 'kcore_min', 'clustering_mean'],
      dtype='object')

In [ ]:
def add_common_neighbors_score(df, G, col_name='score_cn'):

    # Adiciona a pontuação de Common Neighbors (nº de vizinhos em comum) ao dataframe,
    #usando a mesma estratégia de primeiro adicionar as métricas a um dicionário.

    score_dict = {
        (u, v): len(list(nx.common_neighbors(G, u, v)))
        for u, v in zip(df['node1'], df['node2'])
    }

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
def evaluate_auc(df, score_column, label_column='label'):

    auc = roc_auc_score(df[label_column], df[score_column])
    print(f"AUC for {score_column}: {auc:.4f}")
    return {"auc": float(auc)}

def get_duration(start, end):
  return round(end-start, 6)

In [ ]:
start = time.time()
df_cn = add_common_neighbors_score(df_test, G_train)
end = time.time()
model_name = 'score_cn'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_cn: 0.9609


#### Common Neighbors Based - Hub Promoted Index (HPI)

In [ ]:
def add_hpi_score(df, G, col_name='score_hpi'):
    #HPI = |common_neighbors(u, v)| / min(degree(u), degree(v))

    score_dict = {
        (u, v): len(list(nx.common_neighbors(G, u, v))) / min(G.degree[u], G.degree[v])
        for u, v in zip(df['node1'], df['node2'])
        if min(G.degree[u], G.degree[v]) > 0
    }

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_hpi_score(df_test, G_train)
end = time.time()
model_name = 'score_hpi'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_hpi: 0.9555


#### Common Neighbors Based - Preferential Attachment (PA)

In [ ]:
def add_pa_score(df, G, col_name='score_pa'):
    #PA = degree(u) * degree(v)

    score_dict = {
        (u, v): G.degree[u] * G.degree[v]
        for u, v in zip(df['node1'], df['node2'])
    }

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df


In [ ]:
start = time.time()
df_cn = add_pa_score(df_test, G_train)
end = time.time()
model_name = 'score_pa'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_pa: 0.7672


#### Common Neighbors Based - Salton Index (SI)

In [ ]:
def add_salton_score(df, G, col_name='score_salton'):
    #Salton = |common_neighbors(u, v)| / sqrt(degree(u) * degree(v))

    score_dict = {
        (u, v): len(list(nx.common_neighbors(G, u, v))) /
                ( (G.degree[u] * G.degree[v]) ** 0.5 )
        for u, v in zip(df['node1'], df['node2'])
        if G.degree[u] > 0 and G.degree[v] > 0
    }

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_salton_score(df_test, G_train)
end = time.time()
model_name = 'score_salton'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_salton: 0.9553


#### Common Neighbors Based - Jaccard Index (JI)

In [ ]:
def add_jaccard_score(df, G, col_name='score_jaccard'):
    #Jaccard = |common_neighbors(u, v)| / |neighbors(u) ∪ neighbors(v)|

    score_dict = {}
    for u, v in zip(df['node1'], df['node2']):
        neighbors_u = set(G.neighbors(u))
        neighbors_v = set(G.neighbors(v))
        intersection = neighbors_u & neighbors_v
        union = neighbors_u | neighbors_v
        score = len(intersection) / len(union) if len(union) > 0 else 0
        score_dict[(u, v)] = score

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_jaccard_score(df_test, G_train)
end = time.time()
model_name = 'score_jaccard'
result = evaluate_auc(df_cn, 'score_jaccard')
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_jaccard: 0.9543


#### Common Neighbors Based - Sorensen Index (SOI)

In [ ]:
def add_sorensen_score(df, G, col_name='score_sorensen'):
    #Sorensen = 2 * |common_neighbors(u, v)| / (degree(u) + degree(v))

    score_dict = {}
    for u, v in zip(df['node1'], df['node2']):
        deg_u = G.degree[u]
        deg_v = G.degree[v]
        common = len(list(nx.common_neighbors(G, u, v)))
        denominator = deg_u + deg_v
        score = (2 * common / denominator) if denominator > 0 else 0
        score_dict[(u, v)] = score

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_sorensen_score(df_test, G_train)
end = time.time()
model_name = 'score_sorensen'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_sorensen: 0.9543


#### Common Neighbors Based - Hub Depressed Index (HDI)

In [ ]:
def add_hdi_score(df, G, col_name='score_hdi'):
    #HDI = |common_neighbors(u, v)| / max(degree(u), degree(v))

    score_dict = {
        (u, v): len(list(nx.common_neighbors(G, u, v))) / max(G.degree[u], G.degree[v])
        for u, v in zip(df['node1'], df['node2'])
        if max(G.degree[u], G.degree[v]) > 0
    }

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_hdi_score(df_test, G_train)
end = time.time()
model_name = 'score_hdi'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_hdi: 0.9536


#### Common Neighbors Based - Leicht-Holme-Newman Index (LLHN)

In [ ]:
def add_llhn_score(df, G, col_name='score_llhn'):
    #LLHN = |common_neighbors(u, v)| / (degree(u) * degree(v))

    score_dict = {}
    for u, v in zip(df['node1'], df['node2']):
        deg_u = G.degree[u]
        deg_v = G.degree[v]
        denom = deg_u * deg_v
        if denom > 0:
            common = len(list(nx.common_neighbors(G, u, v)))
            score_dict[(u, v)] = common / denom

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_llhn_score(df_test, G_train)
end = time.time()
model_name = 'score_llhn'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_llhn: 0.9483


#### Common Neighbors Based - Resource Allocation Index (RA)

In [ ]:
def add_ra_score(df, G, col_name='score_ra'):
    #RA = soma de 1 / degree(w) para cada vizinho comum w de u e v
    score_dict = {}
    for u, v in zip(df['node1'], df['node2']):
        common_neighbors = set(nx.common_neighbors(G, u, v))
        score = sum(1 / G.degree[w] for w in common_neighbors if G.degree[w] > 0)
        score_dict[(u, v)] = score

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_ra_score(df_test, G_train)
end = time.time()
model_name = 'score_ra'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_ra: 0.9679


#### Common Neighbors Based - 	Adamic-Adar Index (AA)

In [ ]:
def add_aa_score(df, G, col_name='score_aa'):
    #AA = soma de 1 / log(degree(w)) para cada vizinho comum w de u e v
    score_dict = {}
    for u, v in zip(df['node1'], df['node2']):
        common_neighbors = set(nx.common_neighbors(G, u, v))
        score = sum(1 / math.log(G.degree[w]) for w in common_neighbors if G.degree[w] > 1)
        score_dict[(u, v)] = score

    df[col_name] = df.apply(
        lambda row: score_dict.get((row['node1'], row['node2']),
                                   score_dict.get((row['node2'], row['node1']), 0)),
        axis=1
    )

    return df

In [ ]:
start = time.time()
df_cn = add_aa_score(df_test, G_train)
end = time.time()
model_name = 'score_aa'
result = evaluate_auc(df_cn, model_name)
results.append({**result, "model": model_name, 'type': 'common-neighbors-based', 'duration': get_duration(start, end)})

AUC for score_aa: 0.9677


### Path based

The idea: *"If there are many short paths between two nodes, it's more likely that a direct connection between them will eventually form."*

This is also a **heuristic**

#### Unsupervised Approach (Path-Based):

- For each node pair, we calculate a **path-based score**
- We then compare this score with the scores of other pairs in the network.
- We look for the most similar pairs based on score proximity — those with similar path-based scores.
- Next, we check the **labels** of those similar pairs.
- If the majority of these similar pairs are actual links, then the pair we're analyzing is also **likely** to be a real link.



#### Path based - Katz Index Algorithm (KI)

In [ ]:
A = nx.to_numpy_array(G_train)
lambda_max = max(np.linalg.eigvals(A).real)
print(f"Maior autovalor da matriz A: {lambda_max:.4f}")

Maior autovalor da matriz A: 9.9503


In [ ]:
#beta < 1/max_autovalor = 0.105. Portanto, vou usar 0.07

In [ ]:
def add_katz_score(df, G, beta=0.07, max_power=5):

    nodes = list(G.nodes())
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}

    #matriz de adjacência que descreve a rede
    A = nx.to_numpy_array(G, nodelist=nodes)

    katz_matrix = np.zeros_like(A)
    A_power = A.copy()
    for i in range(1, max_power + 1):
        katz_matrix += (beta ** i) * A_power
        A_power = A_power @ A

    scores = []
    for u, v in zip(df['node1'], df['node2']):
        i, j = node_to_idx[u], node_to_idx[v]
        scores.append(katz_matrix[i][j])

    df['score_katz'] = scores
    return df

In [ ]:
start = time.time()
df_test = add_katz_score(df_test, G_train, beta=0.07, max_power=3)
end = time.time()
model_name = 'score_katz'
result = evaluate_auc(df_test, model_name)
results.append({**result, "model": model_name, 'type': 'path-based', 'duration': get_duration(start, end)})

AUC for score_katz: 0.9721


#### Path Based - Random Walk with Restart (RWR)


In [ ]:
def add_rwr_score(df, G, alpha=0.85, max_iter=100, tol=1e-6):
    #r = (1 - α) * (I - αP)^(-1) * e

    nodes = list(G.nodes())
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}
    idx_to_node = {idx: node for node, idx in node_to_idx.items()}
    n = len(nodes)

    # Matriz de adjacência normalizada por coluna
    A = nx.to_numpy_array(G, nodelist=nodes)
    col_sum = A.sum(axis=0)
    col_sum[col_sum == 0] = 1
    P = A / col_sum  # matriz de transição column-normalized

    #RWR para cada nó de origem
    rwr_vectors = {}
    I = np.eye(n)
    for idx in range(n):
        e = np.zeros(n)
        e[idx] = 1
        M = I - alpha * P
        try:
            r = np.linalg.solve(M, (1 - alpha) * e)
        except np.linalg.LinAlgError:
            r = np.zeros(n)
        rwr_vectors[idx] = r

    scores = []
    for u, v in zip(df['node1'], df['node2']):
        i, j = node_to_idx[u], node_to_idx[v]
        scores.append(rwr_vectors[i][j])

    df['score_rwr'] = scores
    return df


In [ ]:
start = time.time()
df_test = add_rwr_score(df_test, G_train)
end = time.time()
model_name = 'score_rwr'
result = evaluate_auc(df_test, model_name)
results.append({**result, "model": model_name, 'type': 'path-based', 'duration': get_duration(start, end)})

AUC for score_rwr: 0.9618


### Classifier Based

The idea: *"Can we train a model to recognize patterns in node pairs that are likely to be connected?"*

Here, we treat link prediction as a **binary classification problem** using supervised learning.

#### Supervised Approach:

- For each pair of users, we extract a set of **features**, such as structural features, node features, combined features.
- This feature table (a dataframe) is used to train a classification model, so the model learns patterns that distinguish likely links from non-links.
- Once trained, it can predict the **probability** of a link forming between any given pair of nodes.


In [ ]:
features = [
    'common_neighbors', 'grau_soma', 'closeness_soma',
    'betweenness_max', 'eigenvector_prod', 'kcore_min', 'clustering_mean'
]

#### Classifier Based - Random Forest (RF)

In [ ]:
model = RandomForestClassifier()
model.fit(df_train[features], df_train['label'])

RandomForestClassifier()

In [ ]:
def classifier_prediction(model, df, feature_cols, label_col='label'):
    X = df[feature_cols]
    y_true = df[label_col]
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1]  # probabilidade da classe 1
    return y_true, y_pred, y_prob

def evaluate_classifier(model, df, feature_cols, label_col='label'):
    start = time.time()
    y_true, y_pred, y_prob = classifier_prediction(model, df, feature_cols)
    end = time.time()

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)

    print(f"Acurácia:  {acc:.4f}")
    print(f"Precisão:  {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"AUC ROC:   {auc:.4f}")

    return {"accuracy": acc, "precision": prec, "recall": rec, "auc": float(auc), "duration": get_duration(start, end)}


In [ ]:
model_name = 'random-forest'
result = evaluate_classifier(model, df_test, feature_cols=features)
results.append({**result, "model": model_name, 'type': 'classifier-based'})

Acurácia:  0.8439
Precisão:  0.8839
Recall:    0.7919
AUC ROC:   0.9568


#### Classifier Based - XGBoost (XG)

In [ ]:
model = XGBClassifier(
    n_estimators=100,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(df_train[features], df_train['label'])

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[22:22:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
model_name = 'xgboost'
result = evaluate_classifier(model, df_test, feature_cols=features)
results.append({**result, "model": model_name, 'type': 'classifier-based'})

Acurácia:  0.8642
Precisão:  0.9145
Recall:    0.8035
AUC ROC:   0.9554


#### Classifier Based - Support Vector Machine for Classification (SVC)

In [ ]:
model = SVC(kernel='linear', C=1.0, probability=True, random_state=42)
model.fit(df_train[features], df_train['label'])

SVC(kernel='linear', probability=True, random_state=42)

In [ ]:
model_name = 'svc'
result = evaluate_classifier(model, df_test, feature_cols=features)
results.append({**result, "model": model_name, 'type': 'classifier-based'})

Acurácia:  0.9509
Precisão:  0.9382
Recall:    0.9653
AUC ROC:   0.9856


####  Classifier Based - Regressão Logística (LR)

In [ ]:
def regression_prediction(X_test):
    # Predições
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    return y_pred, y_prob

def evaluate_reg(model, df_test):

    X_test = df_test[features]
    y_test = df_test['label']

    start = time.time()
    y_pred, y_prob = regression_prediction(X_test)
    end = time.time()

    # Avaliação
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"Logistic Regression Results:")
    print(f"Acurácia:  {acc:.4f}")
    print(f"Precisão:  {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"AUC ROC:   {auc:.4f}")

    return {"accuracy": acc, "precision": prec, "recall": rec, "auc": float(auc), "duration": get_duration(start, end)}

In [ ]:
def train_lr(df_train):

    X_train = df_train[features]
    y_train = df_train['label']

    # podemos tentar ajustar hiperparâmetros aqui, talvez uma regularização, e ver se melhora desempenho (que já tá ok, mas pq a rede é meio toy)
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    return model

In [ ]:
model_name = 'logistic-regression'
model_lr = train_lr(df_train)
result = evaluate_reg(model_lr, df_test)
results.append({**result, "model": model_name, 'type': 'classifier-based'})

Logistic Regression Results:
Acurácia:  0.9509
Precisão:  0.9382
Recall:    0.9653
AUC ROC:   0.9856


####  Classifier Based - Gaussian Naive Bayes (NB)

In [ ]:
#não sei se é muito adequado, pq ele pressupõe independência entre as variáveis e tudo que as conexões de uma rede não são é independentes
#tem as redes bayesianas, que tentam desviar desse problema. cheguei a tentar implementar mas não ficou bom
#teve pior desempenho de todos os modelos

In [ ]:
def train_nb(df_train):

    X_train = df_train[features]
    y_train = df_train['label']

    model = GaussianNB()
    model.fit(X_train, y_train)

    return model

In [ ]:
model_name = 'gaussian-naive-bayes'
model_nb = train_nb(df_train)
result = evaluate_reg(model_nb, df_test)
results.append({**result, "model": model_name, 'type': 'classifier-based'})

Logistic Regression Results:
Acurácia:  0.9509
Precisão:  0.9382
Recall:    0.9653
AUC ROC:   0.9856


### Classifier based + common neighbors measurements

In [ ]:
df_cn_train, df_cn_test = train_test_split(df_cn, test_size=0.3, random_state=49)

In [ ]:
features = ['common_neighbors', 'grau_soma', 'closeness_soma',
       'betweenness_max', 'eigenvector_prod', 'kcore_min', 'clustering_mean',
       'score_cn', 'score_hpi', 'score_pa', 'score_salton', 'score_jaccard',
       'score_sorensen', 'score_hdi', 'score_llhn', 'score_ra', 'score_aa']


####  Classifier Based + CN - Random Forest (RF)

In [ ]:
model = RandomForestClassifier()
model.fit(df_cn_train[features], df_cn_train['label'])

RandomForestClassifier()

In [ ]:
model_name = 'random-forest-cn'
result = evaluate_classifier(model, df_cn_test, feature_cols=features)
results.append({**result, "model": model_name, 'type': 'classifier-and-cn-based'})

Acurácia:  0.9712
Precisão:  0.9615
Recall:    0.9804
AUC ROC:   0.9954


####  Classifier Based + CN - XGBoost

In [ ]:
model = XGBClassifier(
    n_estimators=100,
    use_label_encoder=False,
    eval_metric='logloss',  #inferno d warning
    random_state=42
)

model.fit(df_cn_train[features], df_cn_train['label'])

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[22:22:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
model_name = 'xgboost-cn'
result = evaluate_classifier(model, df_cn_test, feature_cols=features)
results.append({**result, "model": model_name, 'type': 'classifier-and-cn-based'})

Acurácia:  0.9519
Precisão:  0.9259
Recall:    0.9804
AUC ROC:   0.9845


####  Classifier Based + CN - Support Vector Machine for Classification (SVC)

In [ ]:
model = SVC(kernel='linear', C=1.0, probability=True, random_state=42)
model.fit(df_cn_train[features], df_cn_train['label'])

SVC(kernel='linear', probability=True, random_state=42)

In [ ]:
model_name = 'svc-cn'
result = evaluate_classifier(model, df_cn_test, feature_cols=features)
results.append({**result, "model": model_name, 'type': 'classifier-and-cn-based'})

Acurácia:  0.9712
Precisão:  0.9615
Recall:    0.9804
AUC ROC:   0.9834


####  Classifier Based + CN - Regressão Logística (LR)

In [ ]:
model_name = 'logistic-regression-cn'
model_lr = train_lr(df_cn_train)
result = evaluate_reg(model_lr, df_cn_test)
results.append({**result, "model": model_name, 'type': 'classifier-and-cn-based'})

Logistic Regression Results:
Acurácia:  0.9712
Precisão:  0.9615
Recall:    0.9804
AUC ROC:   0.9834


#### Classifier Based + CN - Gaussian Naive Bayes (NB)

In [ ]:
model_name = 'gaussian-naive-bayes-cn'
model_nb = train_nb(df_cn_train)
result = evaluate_reg(model_nb, df_cn_test)
results.append({**result, "model": model_name, 'type': 'classifier-and-cn-based'})

Logistic Regression Results:
Acurácia:  0.9712
Precisão:  0.9615
Recall:    0.9804
AUC ROC:   0.9834


###Probabilistic and Statistical Models Based

The idea: *"Can we model the probability of a link forming based on statistical patterns observed in the network?"*

In this approach, we don't just look at similarity scores or train a machine learning model. Instead, we use **statistical reasoning** to estimate how likely it is that two nodes should be connected, based on the structure of the network and certain assumptions about how links are formed.

#### How it works:

- We define a **probabilistic model** that describes how links are expected to appear in the network.
- Unlike heuristic or machine learning approaches, probabilistic models often provide a **mathematical explanation** for why a link is likely — not just a prediction.

This approach is especially useful when we want to **understand the underlying mechanisms** of network growth, or when working with **incomplete or noisy data**, where uncertainty needs to be modeled explicitly.

#### Probabilistic and Statistical Models Based - Stochastic Block Model (SBM)

In [ ]:
import community as community_louvain

In [ ]:
def compute_block_probabilities(G, partition):
    #calcula a probabilidade empírica de conexão entre comunidades.

    block_edges = defaultdict(int)
    block_counts = defaultdict(int)

    for u, v in G.edges():
        c_u = partition[u]
        c_v = partition[v]
        if c_u > c_v:
            c_u, c_v = c_v, c_u
        block_edges[(c_u, c_v)] += 1

    nodes_in_block = defaultdict(set)
    for node, comm in partition.items():
        nodes_in_block[comm].add(node)

    for b1 in nodes_in_block:
        for b2 in nodes_in_block:
            if b1 > b2:
                continue
            n1 = len(nodes_in_block[b1])
            n2 = len(nodes_in_block[b2]) if b1 != b2 else n1 * (n1 - 1) // 2
            block_counts[(b1, b2)] = n1 * n2 if b1 != b2 else n2

    #calcular probabilidade
    prob = {}
    for key in block_counts:
        e = block_edges.get(key, 0)
        n = block_counts[key]
        prob[key] = e / n if n > 0 else 0

    return prob

In [ ]:
def assign_sbm_score(df, partition, block_prob, col_name='score_sbm'):

    #atribui a cada par (u, v) o score da probabilidade de suas comunidades se conectarem.

    scores = []
    for u, v in zip(df['node1'], df['node2']):
        c_u = partition[u]
        c_v = partition[v]
        if c_u > c_v:
            c_u, c_v = c_v, c_u
        scores.append(block_prob.get((c_u, c_v), 0))
    df[col_name] = scores
    return df

In [ ]:
start = time.time()
#detectar comunidades no G_train
partition = community_louvain.best_partition(G_train)

#calcular probabilidades entre blocos
block_prob = compute_block_probabilities(G_train, partition)

#aplicar no df_test
df_test = assign_sbm_score(df_test, partition, block_prob)
end = time.time()

#avaliar com AUC
auc = roc_auc_score(df_test['label'], df_test['score_sbm'])
print(f"SBM AUC: {auc:.4f}")
results.append({"model": 'stochastic-block', 'type': 'statistical-based', 'auc': float(auc), 'duration': get_duration(start, end)})

SBM AUC: 0.9379


# Conclusão

In [ ]:
results = pd.DataFrame(results)
results

,auc,model,type,duration,accuracy,precision,recall
0,0.960924,score_cn,common-neighbors-based,0.006470,NaN,NaN,NaN
1,0.955545,score_hpi,common-neighbors-based,0.008678,NaN,NaN,NaN
2,0.767249,score_pa,common-neighbors-based,0.005667,NaN,NaN,NaN
3,0.955311,score_salton,common-neighbors-based,0.006085,NaN,NaN,NaN
4,0.954342,score_jaccard,common-neighbors-based,0.007991,NaN,NaN,NaN
5,0.954342,score_sorensen,common-neighbors-based,0.006018,NaN,NaN,NaN
6,0.953607,score_hdi,common-neighbors-based,0.006012,NaN,NaN,NaN
7,0.948261,score_llhn,common-neighbors-based,0.008035,NaN,NaN,NaN
8,0.967874,score_ra,common-neighbors-based,0.006729,NaN,NaN,NaN
9,0.967740,score_aa,common-neighbors-based,0.009956,NaN,NaN,NaN


## Comparação entre Classifier-based e Classifier-based + Common neighbors measurements

In [ ]:
import plotly.graph_objects as go

metrics = ['accuracy', 'precision', 'recall', 'auc', 'duration']
types = ['classifier-based', 'classifier-and-cn-based']

# Seu dataframe filtrado e agrupado
df_filtered = results[results['type'].isin(types)].copy()
df_filtered['base_model'] = df_filtered['model'].str.replace(r'-cn$', '', regex=True)
df_metrics = df_filtered[['base_model', 'type'] + metrics].drop_duplicates()
df_grouped = df_metrics.groupby(['base_model', 'type'], as_index=False).mean()

base_models = df_grouped['base_model'].unique()

fig = go.Figure()

for i, metric in enumerate(metrics):
    for j, t in enumerate(types):
        df_temp = df_grouped[df_grouped['type'] == t]
        visible = True if i == 0 else False

        fig.add_trace(go.Bar(
            x=df_temp['base_model'],
            y=df_temp[metric],
            name=t,
            visible=visible,
        ))

buttons = []
for i, metric in enumerate(metrics):
    visible = [False] * (len(metrics) * len(types))
    for j in range(len(types)):
        visible[i*len(types) + j] = True

    buttons.append(dict(
        label=metric,
        method='update',
        args=[{'visible': visible},
              {'title': f'Comparação de {metric.capitalize()} entre modelos (Classifier vs Classifier + CN)'}]
    ))

fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction='down',
        x=0,
        y=1.15,
        showactive=True,
    )],
    barmode='group',
    xaxis_title='Modelo',
    yaxis_title='Valor',
)

fig.show()

## Comparação entre os modelos Common Neighbors Based

In [ ]:
df_path_based = results[results['type'] == 'common-neighbors-based'].copy()

df_path_based['base_model'] = df_path_based['model'].str.replace(r'-cn$', '', regex=True)
df_grouped = df_path_based.groupby('base_model', as_index=False)['auc'].mean()

fig = px.bar(
    df_grouped,
    x='auc',
    y='base_model',
    orientation='h',
    title='Comparação do AUC entre modelos (Common Neighbors based)',
    labels={'auc': 'AUC', 'base_model': 'Modelo'},
    text='auc'
)


fig.update_layout(yaxis={'categoryorder':'total ascending'}, xaxis_range=[0,1])
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis=dict(range=[0.7, 1])
)
fig.show()

## Comparação entre todos os modelos

In [ ]:
# Agrupar por base_model e type, e calcular média do AUC
df_grouped = results.groupby(['model', 'type'], as_index=False)['auc'].mean()

df_grouped['auc_text'] = df_grouped['auc'].round(4).astype(str)

# Ordenar pelo AUC de forma decrescente
df_grouped = df_grouped.sort_values(by='auc', ascending=False)

# Plotar gráfico de barras horizontal
fig = px.bar(
    df_grouped,
    x='auc',
    y='model',
    color='type',
    orientation='h',
    title='AUC de todos os modelos por tipo',
    labels={'auc': 'AUC', 'model': 'Modelo', 'type': 'Tipo'},
    text='auc_text'
)

# Atualizar layout com ordem decrescente
fig.update_layout(
    yaxis={'categoryorder': 'total descending'},
    xaxis=dict(range=[0.74, 1])  # ajuste conforme necessário
)

fig.show()

### Summary of Insights from the Models and Their Types

We can draw several insights regarding the models and their categories:

- **XGBoost and Random Forest** can be considered relatively weaker models when compared to other classifiers such as Logistic Regression, SVC, and Gaussian Naive Bayes.
  - However, **Random Forest** showed a noticeable improvement when we added **Common Neighbor-based features**. We can assume that these new features allowed the trees to create more meaningful branches, improving performance. In fact, all models showed at least a slight performance boost with the inclusion of neighborhood-based features.

- The best-performing model type overall was **Classifier + Common Neighbor Measurements**.

- The models that achieved the best **AUC** scores were:
  - **Support Vector Classifier (SVC)**
  - **Logistic Regression**
  - **Gaussian Naive Bayes**  
  All three were used in combination with Common Neighbor features. Their performance metrics were:
    - **Accuracy:**  0.9615  
    - **Precision:** 0.9608  
    - **Recall:**    0.9608  
    - **AUC ROC:**   0.9941

- Among these top models, **Logistic Regression** had the fastest processing time at **5,311 ms**. However, this is based on a single run, and more iterations are needed for statistically reliable timing.  
  Additionally, given the similarity in performance among the top models, other factors can influence model selection — such as the **final objective** (e.g., inference vs. prediction) and the **context** of the problem (where one model’s logic may align better with domain needs).

- Interestingly, even the models based solely on **Common Neighbor heuristics** performed competitively. This aligns with expectations: according to the reference text for this project, algorithms that follow the Common Neighbor paradigm tend to work well on **co-authorship networks**. As the authors state, *"authors who belong to the same organization have a high probability of publishing papers together."*

- In addition, **path-based algorithms** also performed well, which is consistent with the findings in the reference work. According to the authors, this is because *"there is a considerable portion of links between different organizations."*
